# Prerequisites 

To run this pipeline, you need both the `jsonschema` and `ogc-na` Python modules installed in your environment (for example, by running `pip install jsonschema ogc-na`); `rdflib` is needed as well, but it is transitively provided by `ogc-na`. If you are running this notebook inside the provided Docker image, good news: that has already been take care of for you.

In [ ]:
from ogc.na.util import load_yaml
import jsonschema
import sys
from pathlib import Path
import json
from rdflib import Graph

You should also know the path of a local OGC Building Blocks repository (either the central repository or a submodule) on your system; if you are using the provided Docker image, you should have already mounted it as a volume using a `-v "/full/path/to/bbr:/notebook/bbr"` parameter for the `docker run` command.

Update the following variable accordingly:

In [ ]:
BBR_LOCATION = Path('/notebook/bbr')

# Setting up the validation environment 

`jsonschema` does not work with JSON schemas written in YAML, so we are going to import our own `RefResolver` to support that.

In [ ]:
from ogc.na.validation import YamlSchemaRefResolver

We are also going to create a convenience function to look up schemas from a building block identifier and return a `jsonschema` validator:

In [ ]:
SCHEMA_LOCATION = BBR_LOCATION / 'annotated-schemas'
if not SCHEMA_LOCATION.is_dir():
    SCHEMA_LOCATION = BBR_LOCATION / 'build/annotated'
if not SCHEMA_LOCATION.is_dir():
    raise ValueError("Cannot find annotated schemas")
else:
    print("Annotated schemas found at", SCHEMA_LOCATION)

def get_subpaths(bblock_id):
    # Build a path from splitting id on every '.' and removing the first component ("ogc")
    return bblock_id.split('.')[1:]

def get_validator(bblock_id):
    bblock_subpaths = get_subpaths(bblock_id)
    # Get the location of the schema inside SCHEMA_LOCATION
    schema_location = SCHEMA_LOCATION / Path(*bblock_subpaths) / 'schema.yaml'
    # Load schema contents and create validator
    schema = load_yaml(filename=schema_location)
    validator_cls = jsonschema.validators.validator_for(schema)
    validator_cls.check_schema(schema)
    resolver = YamlSchemaRefResolver(base_uri=schema_location.as_uri(), referrer=True)
    return validator_cls(schema, resolver=resolver)

# Validating an object

The next step is to create (or load) a JSON object to validate it against a Building Block schema. Modify the following code to suit your needs:

In [ ]:
# Create a JSON object from a dictionary or a list
# Alternatively you may load it from a JSON or YAML file; load_yaml() supports both:
#   json_object = load_yaml(filename='/path/to/file.json')
json_object = json.loads('''{
    "type": "Feature",
    "properties": {
        "name": "Santander"
    },
    "geometry": {
        "coordinates": [-3.809992773317788, 43.461934688127826],
        "type": "Point"
    }
}''')

# Building block identifier for validation
bblock_id = 'ogc.geo.features.feature'

# Validate
validator = get_validator(bblock_id)
error = jsonschema.exceptions.best_match(validator.iter_errors(json_object))
if error is not None:
    print(f"Found a validation error!\n{error}", file=sys.stderr)
else:
    print("Validation ok")

# Adding JSON-LD context

If our building block provides a JSON-LD context, we can add it to our object and see the resulting RDF:

In [ ]:
jsonld_context_file = SCHEMA_LOCATION / Path(*get_subpaths(bblock_id)) / 'context.jsonld'
if jsonld_context_file.is_file():
    # Load JSON-LD context
    with open(jsonld_context_file) as f:
        jsonld_context = json.load(f)

    # Add @context to json_object
    jsonld_object = {
        '@context': jsonld_context['@context'],
        **json_object
    }

    # Parse JSON-lD object and print Turtle
    g = Graph().parse(data=json.dumps(jsonld_object), format='json-ld')
    print(g.serialize(format='ttl'))
else:
    print("No JSON-LD context found for", bblock_id)